In [ ]:
import numpy as np

import tensorflow as tf

from model import *
from input import NetworkInput
from config import *


from data import createFeaturesDescription


from datetime import datetime

import os
import sys

In [ ]:
write_txt_files=True

from IPython.utils.io import Tee

### Configuration

In [ ]:
batch_size=512
num_steps=32

keep_prob=1
keep_prob_recurrent=1
keep_prob_dense_layer=1

data_augmentation=True
name_train='train'

hidden_size=128
num_layers=2

stride = 2

run_with_papermill=False #if true, verbose mode will be set to 2 (1 line/epoch)

kernel_regularizer_l2=0
recurrent_regularizer_l2=0
use_deltas=False

nb_epochs=30

train_model=True
save_weights=True


load_weights=False
weights_filename="./logdir/2020-10-29_17-50-58-size_3_128/weights/weights_2020-10-29_17-50-58.h5" 

In [ ]:

config=Config(batch_size, num_steps, keep_prob=keep_prob, keep_prob_recurrent=keep_prob_recurrent, 
              hidden_size=hidden_size, num_layers=num_layers, 
              kernel_regularizer_l2=kernel_regularizer_l2, recurrent_regularizer_l2=recurrent_regularizer_l2)
config=completedConfig(config) #take default params for unspecified params


In [ ]:
config

In [ ]:

features_description=createFeaturesDescription(F0=False) #Features RMS, RMS HP


### Input

In [ ]:
languages = ["Danish", "Dutch", "English", "Finnish",
    "French", "German", "Hungarian", "Italian",
    "Japanese", "Korean", "Mandarin", "Polish",
    "Portuguese", "Russian", "Spanish",
    "Swedish", "Turkish", "Estonian", "Arabic", "Czech", "Romanian",
    "Basque", "Catalan"]

#Remove languages with not enough data
#languages.remove("Czech")
#languages.remove("Romanian")

# FIRST VERSION
#languages = ['Danish', 'Russian', 'Mandarin', 'Finnish', 'Dutch', 'English', 'Hungarian', 'Swedish', 
#             'Italian', 'French', 'Japanese', 'German', 'Portuguese', 'Polish', 'Spanish', 'Korean']

sets ={}

set_folds=[0]

sets_folds = {"train" : [0, 1, 2],
        "test":[3,4],
        "test1" : [3],
        "test2" : [4]}

set_name='train'
sets[set_name] = NetworkInput(config, folder='./Scores', 
                     subfolder=["fold_{}/".format(k_fold) for k_fold in sets_folds[set_name]],
            stride=stride, verbose=True,                                    
             languages=languages, name=set_name, features_description=features_description, 
                             data_augmentation=data_augmentation, use_deltas=use_deltas)

set_name='test'
sets[set_name] = NetworkInput(config, folder='./Scores', for_evaluation=True,
        subfolder=["fold_{}/".format(k_fold) for k_fold in sets_folds[set_name]],
            stride=stride, verbose=True,                                    
             languages=None, languages_model=languages, name=set_name,
                features_description=features_description, use_deltas=use_deltas) #autodetect languages

In [ ]:
#TRAIN
networkInput=sets["train"]
set_name="train"

#TEST
#networkInput=sets["test"]
#set_name="test"

In [ ]:
dic_info={lang:{} for lang in languages}

for batch in networkInput.processed_full_batch:
    featuresTensor_strided, language, speaker_joined, database, filename = batch
    for lang, spk, database in zip(language, speaker_joined, database):
        lang=lang.numpy().decode('utf8')
        spk=spk.numpy().decode('utf8')
        database=database.numpy().decode('utf8')
        if not (database in dic_info[lang]):
            dic_info[lang][database]={}
        if not (spk in dic_info[lang][database]):
            dic_info[lang][database][spk]=1
        else:
            dic_info[lang][database][spk]+=1

Number of speakers

In [ ]:
if write_txt_files:
    t=Tee(f'./Files/info_{set_name}_set_speakers.csv')

print(' \t '.join(['Language', 'librivox', 'CommonVoice', 'voxforge', 'WLI', 'tatoeba']))
for lang, dic_database in dic_info.items():
    st=f'{lang}'
    for database in ['librivox', 'CommonVoice', 'voxforge', 'WLI', 'tatoeba']:
        if database in dic_database:
            st+=f' \t {len(dic_database[database])}'  
        else:
            st+=' \t '
    print(st)
    
if write_txt_files:
    t.close()

Number of files

In [ ]:
if write_txt_files:
    t=Tee(f'./Files/info_{set_name}_set_files.csv')

print(' \t '.join(['Language', 'librivox', 'CommonVoice', 'voxforge', 'WLI', 'tatoeba']))
for lang, dic_database in dic_info.items():
    st=f'{lang}'
    for database in ['librivox', 'CommonVoice', 'voxforge', 'WLI', 'tatoeba']:
        if database in dic_database:
            st+=f' \t {sum([nb_files for nb_files in dic_database[database].values()])}'  
        else:
            st+=' \t '
    print(st)
if write_txt_files:
    t.close()